In [ ]:
import requests
import requests_cache
from requests_cache import CachedSession
from datetime import timedelta
from bs4 import BeautifulSoup 

In [ ]:
session = requests_cache.CachedSession('finalCache')
session = CachedSession('finalCache', expire_after=timedelta(hours=24))
finalSoup = BeautifulSoup("", "html5lib")
finalResultData = []
stateList  = []
for j in range(1,30):
    stateData = []    
    for i in range(1,5):
        URL = f"https://results.eci.gov.in/PcResultGenJune2024/statewiseS{j:02}{i}.htm"
        print(URL)
        r = session.get(URL , timeout=10)
        soup = BeautifulSoup(r.content, 'html5lib')
        # finalSoup.append(soup)

        
        ###For checking if the url exits
        data = soup.find('body')
        if type(data) == int:
            continue
        data = data.find('h1')
        if data is None:
            continue
        if "Not Found" in data:
            continue
        # print(soup)
        divs = soup.find("div", class_="page-title").findAll('span')
        stateName = divs[0].text
        if stateName.strip() not in stateList:
            stateList.append(stateName.strip())
        
        table = soup.findAll('tr') 
        
        ls = []
        ls2 = []    
        for row in table[2:]:
            ls.append(row)
            #print(f"{row} herehere")
        for row in  ls:
            tddata = row.findAll('td')
            # print(tddata)
            ls2.append(tddata)
        rawData = [ls2[i] for i in range(0, len(ls2), 8 + 1)]
        rawData = [x for x in rawData if x != []]
        #0 1 2 4 15 17 28 29 
        preFinalRowData = []
        for i in range(len(rawData)):
            preFinalRowData.append([rawData[i][0] , rawData[i][1], rawData[i][2], rawData[i][4], rawData[i][15], rawData[i][17], rawData[i][28], rawData[i][29]])
        
        for row in preFinalRowData:
            temp = []
            for data in row:
                temp.append(data.text)
            stateData.append(temp)
    finalResultData.append(stateData)
## filtering empty lists !!important
filteredFinalResultData = list(filter(None, finalResultData))
res = dict(zip(stateList, filteredFinalResultData ))
print(stateList)

## MYSQL Connection

In [ ]:
import mysql.connector
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="nachiket",
  database="LS2024"
)

mycursor = mydb.cursor()
mycursor1 = mydb.cursor()
# print(stateList)
mydb.start_transaction()
count = 0;
for state in (stateList):
  # if state == 'Gujarat': ## gujrat had an uncontessted win so blank values
  #   continue
  mycursor.execute(f" CREATE TABLE IF NOT EXISTS`{state}` ( Constituency VARCHAR(255),`Const.No` INT,LeadingCandidate VARCHAR(255),LeadingParty VARCHAR(255),TrailingCandidate VARCHAR(255),TrailingParty VARCHAR(255),Margin VARCHAR(255),Status VARCHAR(255),PRIMARY KEY (`Const.No`))")
  sql = f"""
        INSERT INTO `{state}` (Constituency, `Const.No`, LeadingCandidate, LeadingParty, TrailingCandidate, TrailingParty, Margin, Status)
        VALUES (
            NULLIF(%s, ''),
            NULLIF(%s, ''),
            NULLIF(%s, ''),
            NULLIF(%s, ''),
            NULLIF(%s, ''),
            NULLIF(%s, ''),
            NULLIF(%s, ''),
            NULLIF(%s, '')
        )
        ON DUPLICATE KEY UPDATE
            Constituency = VALUES(Constituency),
            LeadingCandidate = VALUES(LeadingCandidate),
            LeadingParty = VALUES(LeadingParty),
            TrailingCandidate = VALUES(TrailingCandidate),
            TrailingParty = VALUES(TrailingParty),
            Margin = VALUES(Margin),
            Status = VALUES(Status)
          """
  # for row in res[state]:
  #   mycursor.execute(sql, row)
  mycursor1.executemany(sql, res[state])
  count = count+1
  if count == 10:
    mydb.commit()
    mydb.start_transaction()
    mycursor1 = mydb.cursor()
    count = 0


mydb.commit()

print(mycursor.rowcount, "record inserted.")

mycursor.close()
mydb.close()
